In [1]:
import numpy as np
import pandas as pd
import pickle
import scipy.stats as ss
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
train = pd.read_csv('Data/train.csv', index_col=0)
train.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [3]:
test = pd.read_csv('Data/test.csv', index_col=0)
test.head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
test_0,11.0656,7.7798,12.9536,9.4292,11.4327,-2.3805,5.8493,18.2675,2.1337,8.8100,...,-2.1556,11.8495,-1.4300,2.4508,13.7112,2.4669,4.3654,10.7200,15.4722,-8.7197
test_1,8.5304,1.2543,11.3047,5.1858,9.1974,-4.0117,6.0196,18.6316,-4.4131,5.9739,...,10.6165,8.8349,0.9403,10.1282,15.5765,0.4773,-1.4852,9.8714,19.1293,-20.9760
test_2,5.4827,-10.3581,10.1407,7.0479,10.2628,9.8052,4.8950,20.2537,1.5233,8.3442,...,-0.7484,10.9935,1.9803,2.1800,12.9813,2.1281,-7.1086,7.0618,19.8956,-23.1794
test_3,8.5374,-1.3222,12.0220,6.5749,8.8458,3.1744,4.9397,20.5660,3.3755,7.4578,...,9.5702,9.0766,1.6580,3.5813,15.1874,3.1656,3.9567,9.2295,13.0168,-4.2108
test_4,11.7058,-0.1327,14.1295,7.7506,9.1035,-8.5848,6.8595,10.6048,2.9890,7.1437,...,4.2259,9.1723,1.2835,3.3778,19.5542,-0.2860,-5.1612,7.2882,13.9260,-9.1846


### Train, validation split

In [4]:
X = train.drop(["target"], axis=1)
y = train["target"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

### Modeling

In [7]:
def xgb_model(folds, param_comb, X_train, y_train, X_val, y_val, save_file):
    """
    
    """
    
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 123)

    xgb = XGBClassifier(n_jobs=1, random_state=123)
    params = {"learning_rate":ss.uniform(0.001,0.5),
             "n_estimators":ss.randint(10,1000),
             "max_depth": ss.randint(3,25),
             "min_child_weight":ss.randint(1,15),
             "gamma": ss.randint(0,10),
             "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             "colsample_bytree": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
             "reg_lambda":ss.uniform(0.05,5),
             "reg_alpha":ss.uniform(0.05,5)}


    model_xgb =  RandomizedSearchCV(estimator=xgb, 
                                    param_distributions=params, 
                                    n_iter=param_comb, 
                                    scoring='roc_auc', 
                                    n_jobs=1, 
                                    cv=skf.split(X_train, y_train), 
                                    verbose=3, 
                                    random_state=123)

    
    model_xgb.fit(X_train, y_train)
    scores = dict()
    scores["train_dataset"] = {
        "Accuracy": np.round(accuracy_score(model_xgb.best_estimator_.predict(X_train), y_train), 4),
        "AUC": np.round(roc_auc_score(y_train, model_xgb.best_estimator_.predict_proba(X_train)[:, 1]), 4)
      }
    scores["validation_dataset"] = {
        "Accuracy":np.round(accuracy_score(model_xgb.best_estimator_.predict(X_val), y_val), 4),
        "AUC": np.round(roc_auc_score(y_val, model_xgb.best_estimator_.predict_proba(X_val)[:, 1]), 4)
      }
    pickle.dump(model_xgb.best_estimator_, open(save_file, 'wb'))
    return model_xgb, scores

In [ ]:
folds = 10
param_comb = 100
xgb = xgb_model(folds, param_comb, X_train, y_train, X_val, y_val, 'xgb.sav')
print(f"XGB: {rf[1]}")
model_xgb = pickle.load(open('xgb.sav', 'rb'))

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] colsample_bytree=1.0, gamma=2, learning_rate=0.11442572678210156, max_depth=9, min_child_weight=2, n_estimators=605, reg_alpha=2.505594667162986, reg_lambda=3.9501388095603955, subsample=0.6 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=1.0, gamma=2, learning_rate=0.11442572678210156, max_depth=9, min_child_weight=2, n_estimators=605, reg_alpha=2.505594667162986, reg_lambda=3.9501388095603955, subsample=0.6, score=0.8798189966549037, total=66.8min
[CV] colsample_bytree=1.0, gamma=2, learning_rate=0.11442572678210156, max_depth=9, min_child_weight=2, n_estimators=605, reg_alpha=2.505594667162986, reg_lambda=3.9501388095603955, subsample=0.6 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 67.0min remaining:    0.0s


In [ ]:
def rf_model(folds, param_comb, X_train, y_train, X_val, y_val, save_file):
    """
    
    """
    
    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 123)
    rf = RandomForestClassifier(n_jobs=1, random_state=123)
    params = {"bootstrap": [True, False],
             "max_depth": ss.randint(3,10),
             "max_features": ['auto', 'sqrt'],
             "min_samples_leaf": ss.randint(1,10),
             "min_samples_split": ss.randint(2,10),
             "n_estimators":ss.randint(10,1000),
             "criterion": ["gini", "entropy"],
             "class_weight": ["balanced", "balanced_subsample", None],
              
             }

    model_rf =  RandomizedSearchCV(estimator=rf, 
                                    param_distributions=params, 
                                    n_iter=param_comb, 
                                    scoring='roc_auc', 
                                    n_jobs=1, 
                                    cv=skf.split(X_train, y_train), 
                                    verbose=3, 
                                    random_state=123)
    model_rf.fit(X_train, y_train)
    scores = dict()
    scores["train_dataset"] = {
        "Accuracy": np.round(accuracy_score(model_rf.best_estimator_.predict(X_train), y_train), 4),
        "AUC": np.round(roc_auc_score(y_train, model_rf.best_estimator_.predict_proba(X_train)[:, 1]), 4)
      }
    scores["validation_dataset"] = {
        "Accuracy":np.round(accuracy_score(model_rf.best_estimator_.predict(X_val), y_val), 4),
        "AUC": np.round(roc_auc_score(y_val, model_rf.best_estimator_.predict_proba(X_val)[:, 1]), 4)
      }
    pickle.dump(model_rf.best_estimator_, open(save_file, 'wb'))
    return model_rf, scores

In [ ]:
folds = 10
param_comb = 100
rf = rf_model(folds, param_comb, X_train, y_train, X_val, y_val, 'rf.sav')
print(f"Random forest: {rf[1]}")
model_rf = pickle.load(open('rf.sav', 'rb'))

In [ ]:
def lgb_model(folds, param_comb, X_train, y_train, X_val, y_val, save_file):
    """

    """

    skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 123)

    lgb = LGBMClassifier(n_jobs=1, random_state=123)
    params = {"num_leaves":ss.randint(2,50),
              "max_depth": [5, 10, 15, -1],
              "learning_rate":ss.uniform(0.001,0.5),
              "n_estimators":ss.randint(10,1000),
              "objective": ["binary"],
              "class_weight": ["balanced", "balanced_subsample", None],
              "subsample": [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              "colsample_bytree": [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
              "reg_lambda":ss.uniform(0.05,5),
              "reg_alpha":ss.uniform(0.05,5),
              "min_child_weight":ss.randint(1,15)}


    model_lgb =  RandomizedSearchCV(estimator=lgb, 
                                    param_distributions=params, 
                                    n_iter=param_comb, 
                                    scoring='roc_auc', 
                                    n_jobs=1, 
                                    cv=skf.split(X_train, y_train), 
                                    verbose=3, 
                                    random_state=123)


    model_lgb.fit(X_train, y_train, )
    scores = dict()
    scores["train_dataset"] = {
        "Accuracy": np.round(accuracy_score(model_lgb.best_estimator_.predict(X_train), y_train), 4),
        "AUC": np.round(roc_auc_score(y_train, model_lgb.best_estimator_.predict_proba(X_train)[:, 1]), 4)
      }
    scores["validation_dataset"] = {
        "Accuracy":np.round(accuracy_score(model_lgb.best_estimator_.predict(X_val), y_val), 4),
        "AUC": np.round(roc_auc_score(y_val, model_lgb.best_estimator_.predict_proba(X_val)[:, 1]), 4)
      }
    pickle.dump(model_lgb.best_estimator_, open(save_file, 'wb'))
    return model_lgb, scores

In [ ]:
folds = 10
param_comb = 100
lgb = lgb_model(folds, param_comb, X_train, y_train, X_val, y_val, 'lgb.sav')
print(f"LGB: {lgb[1]}")
model_lgb = pickle.load(open('lgb.sav', 'rb'))